In [1]:
from dotenv import load_dotenv
import os

# Cargar las variables del archivo .env
load_dotenv()

# Obtener la clave de OpenAI
openai_api_key = os.getenv("OPENAI_API_KEY")

# Verifica que la clave se ha cargado correctamente
if openai_api_key:
    print("La clave de OpenAI se ha cargado correctamente.")
else:
    print("Error al cargar la clave de OpenAI.")

La clave de OpenAI se ha cargado correctamente.


In [2]:
from openai import OpenAI
import os
client=OpenAI(api_key=openai_api_key)
MODEL='gpt-4-turbo'

In [3]:
import yt_dlp

# URL del vídeo de YouTube
url = "https://www.youtube.com/watch?v=G9FRadIkkE0"

# Usar yt-dlp para obtener la información del vídeo
ydl_opts = {}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(url, download=False)
    video_title = info_dict.get('title', None)
    video_views = info_dict.get('view_count', None)
    video_length = info_dict.get('duration', None)
    video_thumbnail = info_dict.get('thumbnail', None)

# Imprimir la información obtenida
print(f"Title: {video_title}")
print(f"Views: {video_views}")
print(f"Length: {video_length} seconds")
print(f"Thumbnail: {video_thumbnail}")


[youtube] Extracting URL: https://www.youtube.com/watch?v=G9FRadIkkE0
[youtube] G9FRadIkkE0: Downloading webpage
[youtube] G9FRadIkkE0: Downloading ios player API JSON
[youtube] G9FRadIkkE0: Downloading mweb player API JSON
[youtube] G9FRadIkkE0: Downloading m3u8 information
Title: Microsoft Recall is MANDATORY
Views: 234578
Length: 299 seconds
Thumbnail: https://i.ytimg.com/vi/G9FRadIkkE0/maxresdefault.jpg


In [4]:
#descargar vídeo con yt-dlp

# Opciones para descargar el vídeo
ydl_opts = {
    'format': 'best',  # Seleccionar la mejor calidad disponible
    'outtmpl': '%(title)s.%(ext)s',  # Guardar el archivo con el título del vídeo
}

# Descargar el vídeo
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])


[youtube] Extracting URL: https://www.youtube.com/watch?v=G9FRadIkkE0
[youtube] G9FRadIkkE0: Downloading webpage
[youtube] G9FRadIkkE0: Downloading ios player API JSON
[youtube] G9FRadIkkE0: Downloading mweb player API JSON
[youtube] G9FRadIkkE0: Downloading m3u8 information
[info] G9FRadIkkE0: Downloading 1 format(s): 18
[download] Destination: Microsoft Recall is MANDATORY.mp4
[download] 100% of   10.79MiB in 00:00:00 at 37.60MiB/s    


In [5]:
import yt_dlp
import re

def descargar_solo_audio(url):
    # Función para limpiar el título del vídeo
    def sanitize_filename(filename):
        # Reemplaza los espacios por guiones bajos
        filename = filename.replace(" ", "_")
        # Elimina caracteres no alfanuméricos, dejando solo letras, números y guiones bajos
        filename = re.sub(r'[^\w-]', '', filename)
        return filename

    print("descargar_solo_audio...")

    # Obtener la información del vídeo y el título
    ydl_opts_info = {}
    with yt_dlp.YoutubeDL(ydl_opts_info) as ydl:
        info_dict = ydl.extract_info(url, download=False)
        video_title = info_dict.get('title', None)

    # Limpiar el título del vídeo
    clean_title = sanitize_filename(video_title)

    # Opciones para descargar solo el audio
    ydl_opts = {
        'format': 'bestaudio/best',  # Descargar la mejor calidad de audio disponible
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',  # Puedes cambiar a 'wav' o 'm4a' si prefieres otro formato
            'preferredquality': '192',  # Calidad de audio
        }],
        'outtmpl': f'{clean_title}.%(ext)s',  # Guardar el archivo con el título limpio
        'quiet': True,  # Esta opción desactiva los mensajes de yt-dlp
        'no_warnings': True  # Desactiva las advertencias
    }

    # Descargar el audio
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

# Devolver el nombre del archivo
    filename = f"{clean_title}.mp3"
    print(f"Descarga de audio completa. Archivo guardado como: {filename}")
    return filename
# Ejemplo de uso:
# descargar_solo_audio("https://www.youtube.com/watch?v=G9FRadIkkE0")


In [ ]:
descargar_solo_audio("https://www.youtube.com/watch?v=wRsX_ZkzxvQ")

descargar_solo_audio...


[youtube] Extracting URL: https://www.youtube.com/watch?v=wRsX_ZkzxvQ
[youtube] wRsX_ZkzxvQ: Downloading webpage
[youtube] wRsX_ZkzxvQ: Downloading ios player API JSON
[youtube] wRsX_ZkzxvQ: Downloading mweb player API JSON
[youtube] wRsX_ZkzxvQ: Downloading m3u8 information


In [4]:

from pydub import AudioSegment
import os

import os
from pydub import AudioSegment

# Función para dividir el audio en segmentos de 120 segundos por defecto
def split_audio(file_path, output_dir=".", base_name=None, chunk_length=120):
     # Si no se proporciona base_name, obtenerlo del file_path
    if base_name is None:
        base_name = os.path.splitext(os.path.basename(file_path))[0]  # Extraer nombre de archivo sin extensión
    
    # Cargar el archivo de audio
    audio = AudioSegment.from_mp3(file_path)
    duration_ms = len(audio)  # Duración total del audio en milisegundos
    chunk_length_ms = chunk_length * 1000  # Convertir la longitud de los fragmentos a milisegundos

    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    chunk_files = []

    # Dividir el audio en fragmentos estándar de chunk_length_ms
    for i, start in enumerate(range(0, duration_ms, chunk_length_ms)):
        print(f"\rFragmento {i+1}/{int(duration_ms/chunk_length_ms)}", end='')
        chunk = audio[start:start + chunk_length_ms]
        chunk_name = f"{i:03d}_{base_name}.mp3"
        chunk_path = os.path.join(output_dir, chunk_name)
        chunk.export(chunk_path, format="mp3")
        chunk_files.append(chunk_path)
    
    return chunk_files

# Ejemplo de uso:
# Dividir un archivo de audio en segmentos de 120 segundos y guardarlos en una carpeta
# split_audio("audio.mp3", "output_dir", "segment", chunk_length=120)


def transcribe(audio_file):
    if not os.path.isfile(audio_file):
        print(f"El archivo {audio_file} no existe.")
        return  False
    
    with open(audio_file, 'rb') as f:
        print("Transcribiendo audio...",end='')
        transcript = client.audio.transcriptions.create(
            model='whisper-1',
            file=f
        )
        print("Transcripción completa.")
        name,extension = os.path.splitext(audio_file)
        transcribed_file = f"transcribed_{name}.txt"
        with open(transcribed_file, 'w') as transcription_file:
            transcription_file.write(transcript.text)
    return transcribed_file

        





In [6]:
segments=split_audio("Neuroscientist_TRIPLE_Your_Testosterone_Levels__Andrew_Huberman.mp3")
# segments=split_audio("Dr_Craig_Koniver_Peptide__Hormone_Therapies_for_Health_Performance__Longevity.mp3")

: 